# 🦀 Rust

## Un langage de programmation système de haut niveau


[Pierre-Antoine Champin](https://champin.net/)

[Inria](https://team.inria.fr/wimmics/) – [W3C](https://www.w3.org/)

## Rust en deux mots

Rust est un langage compilé, fortement typé, open-source,
mettant l'accent sur

* la performance
* la fiabilité
* la productivité

ainsi que sur la gestion sûre de la mémoire et du parallélisme.

https://rust-lang.org

### Historique

* 2006: Graydon Hoare commence à travailler sur Rust
* 2009: Mozilla adopte le projet
* 2015: Version 1.0 du compilateur
* 2021: Création de la [Fondation Rust](https://foundation.rust-lang.org/)

L'objectif de Rust n'est *pas* d'être innovant,
mais au contraire de s'appuyer sur des concepts anciens ("rouillés") et éprouvés.

(même si certains n'ont pas été déployés de façon *mainstream*)

## Sémantique des affectations

In [2]:
let v1: Vec<i32> = vec![22, 44, 66];
println!("v1 = {:?}", &v1);
let v2 = v1;
println!("v2 = {:?}", &v2);

//println!("v1 = {:?}", &v1);

v1 = [22, 44, 66]
v2 = [22, 44, 66]


* Dans la plupart des langages de programmation,
l'affectation a une sémantique de **copie**.

  + Après l'affectation, v1 et v2 contiennent « la même chose ».
     


* En Rust, l'affectation a *par défaut* une sémantique de **déplacement** (*move*).

  + Conceptuellement, l'affectation *déplace* la valeur de v1 dans v2.
  + Après l'affectation, v1 est considérée comme non-initialisée.

### Pourquoi est-ce une bonne chose ?

In [3]:
let v1: Vec<i32> = vec![22, 44, 66];

<div style="text-align: center">
<img alt="String en mémoire" src="images/trpl04-01.svg" style="display: inline; width: 20em;">
</div>

In [4]:
let v1: Vec<i32> = vec![22, 44, 66];
let v2 = v1;
// Si Rust avait une sémantique de copie :

<div style="text-align: center"> <img alt="Shallow copy" src="images/trpl04-02.svg" style="display: inline; width: 20em;"> </div>

In [5]:
let v1: Vec<i32> = vec![22, 44, 66];
let mut v2 = v1;
v2.push(88);
v2.push(99);
// Si Rust avait une sémantique de copie :

<div style="text-align: center"> <img alt="Shallow copy" src="images/trpl04-02bis.svg" style="display: inline; width: 20em;"> </div>

In [6]:
let v1: Vec<i32> = vec![22, 44, 66];
let v2 = v1;
// Avec la sémantique de déplacement de Rust

<div style="text-align: center"> <img alt="Shallow copy" src="images/trpl04-04pa.svg" style="display: inline; width: 20em;"> </div>

### Copie (profonde) explicite

In [7]:
let v1: Vec<i32> = vec![22, 44, 66];
let v2 = v1.clone();
println!("v1={:?} v2={:?}", &v1, &v2);

v1=[22, 44, 66] v2=[22, 44, 66]


<div style="text-align: center">
<img alt="Copie profonde" src="images/trpl04-03.svg" style="display: inline; width: 20em;">
</div>

### Copie implicite

Bien que la sémantique *move* soit la règle,
il existe des exceptions.

En particulier, les affectations pour tous les types de base (entiers, flottants, booléens...)
ont la sémantique *copy*,
car le compilateur sait que cette copie est sûre (*safe*).

In [8]:
let x = 42;
let y = x; // copy instead of move
println!("x={} y={}", x, y);

x=42 y=42


Plus généralement,
tout type implémentant le [trait `Copy`](https://doc.rust-lang.org/stable/std/marker/trait.Copy.html)
utilise la sémantique *copy*.

Il est possible de faite implémenter `Copy` aux types définis par nous, mais ce n'est pas toujours possible ni judicieux (example : le type `Vec`).

## Propriété (*ownership*)

* Toute donnée est la **propriété** d'exactement une variable (ou champ de structure).

* Conceptuellement, l'affectation d'une variable à une autre **déplace** (*move*) les données,
  et **transfère** la propriété à la nouvelle variable.
  
* Lorsqu'une variable disparait,
  (i.e. lorsqu'on quitte son scope),
  si elle est encore propriétaire de ses données,
  celles-ci sont **libérées**.

<div style="text-align: center">
<img alt="String en mémoire" src="images/trpl04-01.svg" style="display: inline; width: 20em;">
</div>

* la variable *v1* est propriétaire du bloc de gauche
* le champ *ptr* du bloc de gauche est propriétaire du bloc de droite


* lorsque *v1* disparait, on libère le bloc de gauche, ce qui entraîne la libération du bloc de droite.

In [9]:
fn fibo(nb: usize) -> Vec<i32> {
    let mut v1 = vec![];                  // 1. Le vecteur est créé ici
    for i in 0..nb {
        if i < 2 { v1.push(1); }
        else     { v1.push(v1[i-2] + v1[i-1]); }
    }
    v1                                    // 2. Sa propriété est transmise au code appelent
}

fn somme(v2: Vec<i32>) -> i32 {
    v2.into_iter().sum()
}                                         // 5. La variable 'v2' arrive au bout de son scope,
                                          //    et elle possède toujours le vecteur,
                                          //    qui est donc libéré.

{ 
    let n = 7;
    let mut v3 = fibo(n);                 // 3. Le vecteur est récupérée depuis la fonc. fibo
    println!("v3={:?}", &v3);
    let s = somme(v3);                    // 4. Le vecteur est transférée à la fonc. somme
    println!("la somme des {} premiers termes vaut {}", n, s);
    // println!("{:?}", &v3); // NE COMPILE PAS
};

v3=[1, 1, 2, 3, 5, 8, 13]
la somme des 7 premiers termes vaut 33


En C, C++, ...
- la libération des données du vecteur est de la responsabilité du développeur,
- ce qui peut donner lieu à des erreurs (mémoire libérée trop tôt, trop tard, jamais...).

En Java, Python, ...
- le ramasse-miettes (*garbage collector*) se charge de libérer la mémoire lorsqu'elle n'est plus utilisée,
- ce qui peut entraîner des problèmes de performances.

En Rust
- la notion de propriété permet au compilateur de savoir où dans le programme insérer les instructions de libération de la mémoire
- → performant et sûr.

## Emprunts (*borrow*)

* Dans certaines situations, on souhaite passer une valeur *sans* transférer sa propriété
* On parle alors d'**emprunt**

In [10]:
fn fibo(nb: usize) -> Vec<i32> {
    let mut v1 = vec![];
    for i in 0..nb {
        if i < 2 { v1.push(1); }
        else     { v1.push(v1[i-2] + v1[i-1]); }
    }
    v1
}

fn somme(v2: Vec<i32>) -> i32 {
    v2.into_iter().sum()
}

{ 
    let n = 7;
    let mut v3 = fibo(n);
    println!("v3={:?}", &v3);
    let s = somme(v3);
    println!("la somme des {} premiers termes vaut {}", n, s);
    println!("{:?}", &v3);
};

Error: borrow of moved value: `v3`

Pour tout type `T`, les variables de type `&T` sont des **références** vers un `T`.

* Physiquement, les références sont des pointeurs.
* Conceptuellement, les références ne *possèdent pas* les données vers lesquelles elles pointent,
* La disparition de la référence n'entraîne donc pas la libération des données.


<div style="text-align: center">
<img alt="String en mémoire" src="images/trpl04-05pa.svg" style="display: inline; width: 30em;">
</div>

* En conséquence, plusieurs références vers les même données peuvent *co-exister*.

In [11]:
let mut v1: Vec<i32> = vec![22, 44, 66];
{
    let v2 = &v1;
    let v3 = &v1;
    println!("v1={:?} v2={:?} v3={:?}", &v1, &v2, &v3);
}
println!("v1={:?}", &v1);

v1=[22, 44, 66] v2=[22, 44, 66] v3=[22, 44, 66]
v1=[22, 44, 66]


NB : Les références de type `&T` sont immutables.

In [12]:
let mut v1: Vec<i32> = vec![22];
v1.push(44);
{
    let v2 = &v1;
    // v2.push(66); // NE COMPILE PAS
    println!("v1={:?} v2={:?}", &v1, &v2);
}
v1.push(88);
println!("v1={:?}", &v1);

v1=[22, 44] v2=[22, 44]
v1=[22, 44, 88]


Pour que l'exemple ci-dessus fonctionne, il faut utiliser une **référence mutable**,
de type `&mut T`.

In [13]:
let mut v1: Vec<i32> = vec![22];
v1.push(44);
{
    let v2 = &mut v1;
    v2.push(66);
}
v1.push(88);
println!("v1={:?}", &v1);

v1=[22, 44, 66, 88]


### Règles sur les références

On ne peut pas avoir en même temps

* plusieurs références mutables, ni
* une référence mutable et une (ou plusieurs) référence(s) immutable(s).

En effet, il faut éviter

* que deux parties du code modifient les mêmes données simultanément, et
* qu'une partie du code lise des données en cours de modification.

In [14]:
let mut v1: Vec<i32> = vec![22];
v1.push(44);
{
    let v2 = &mut v1;
    //v1.push(66); // NE COMPILE PAS
    //let v3 = &v1; // NE COMPILE PAS
    println!("v2={:?}", &v2);
}
v1.push(88);
println!("v1={:?}", &v1);

v2=[22, 44]
v1=[22, 44, 88]


### Différence conceptuelle entre référence et pointeur


In [15]:
let a = 42;
let b = 41 + 1;
let test = (&a == &b);
test

true

* La « valeur » d'une référence n'est **pas** l'adresse en mémoire, c'est la valeur pointée.

    * Ne pas penser `&T` comme « un pointeur vers un `T` » mais comme « un `T` emprunté à quelqu'un d'autre ».
    * `T` et `&T` restent malgré tout deux types différents...

* En Rust, une référence n'est **jamais** un pointeur null.

## Conclusion

Les concepts de **propriété** et d'**emprunt** permettent au compilateur Rust

* de garantir une gestion sûre et performante de la mémoire,
* mais également du parallélisme.